In [2]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 3.2 MB/s eta 0:00:00


In [3]:
!pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.4 MB/s eta 0:00:00


In [5]:
!pip install groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 2.9 MB/s eta 0:00:00


In [11]:
import os
import re
import numpy as np
import pinecone
from sentence_transformers import SentenceTransformer
from groq import Groq
import torch
from typing import List, Dict, Any, Optional
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

class EnhancedRAGSystem:
    def __init__(self, pinecone_api_key: str, groq_api_key: str, index_name: str,
                 model_name: str = "sentence-transformers/all-mpnet-base-v2"):
        """
        Initialize Enhanced RAG system with Pinecone, Groq, and models.
        """
        # Initialize Pinecone client
        self.pc = pinecone.Pinecone(api_key=pinecone_api_key)
        self.index = self.pc.Index(index_name)

        # Initialize Groq client
        self.groq_client = Groq(api_key=groq_api_key)

        # Check for GPU availability
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {self.device}")

        # Initialize embedding model
        self.embedding_model = SentenceTransformer(model_name).to(self.device)

        # Set parameters
        self.top_k = 1  # Retrieve only the top source

        # Initialize NLP tools with better error handling
        try:
            nltk.download('stopwords', quiet=True)
            nltk.download('wordnet', quiet=True)
            self.stop_words = set(stopwords.words('english'))
            self.lemmatizer = WordNetLemmatizer()
        except Exception as e:
            print(f"Warning: NLTK resource download issue. Error: {e}")
            self.stop_words = {'a', 'an', 'the', 'and', 'or', 'but', 'is', 'are', 'was', 'were', 'to', 'of', 'in', 'for'}
            self.lemmatizer = None

        # Load spaCy model with fallback
        try:
            self.nlp = spacy.load("en_core_web_sm")
        except:
            print("Warning: spaCy model 'en_core_web_sm' not found. Using a simple pipeline.")
            self.nlp = spacy.blank("en")

    def generate_answer(self, query: str, relevant_chunks: List[Dict[str, Any]]) -> str:
        """
        Generate a comprehensive answer using Groq's language model based on the query and relevant chunk.
        """
        if not relevant_chunks:
            return "I couldn't find sufficient information to answer your question. Please try rephrasing or asking a different question."

        # Use the top (most relevant) chunk
        top_chunk = relevant_chunks[0]
        context = top_chunk['text']

        # Prepare a focused prompt for the Groq language model
        prompt = f"""
        You are an intelligent assistant specialized in educational content. Your task is to create a comprehensive, well-structured answer to the user's question using the provided context.

        USER QUESTION:
        {query}

        RELEVANT CONTEXT:
        {context}
        """

        # Use Groq's chat completion API
        try:
            response = self.groq_client.chat.completions.create(
                model="llama3-70b-8192",  # You can change this to other available Groq models
                messages=[
                    {"role": "system", "content": "You are an intelligent assistant specialized in educational content."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=512,
                temperature=0.7,
                top_p=0.95
            )

            # Extract the answer from the response
            answer = response.choices[0].message.content.strip()
            return answer

        except Exception as e:
            print(f"Error generating answer with Groq: {e}")
            return "I encountered an error while generating the answer. Please try again."

    # Rest of the methods remain the same as in the original code
    def extract_transcript(self, video_id: str) -> Optional[str]:
        """
        Extract transcript from a YouTube video.

        Args:
            video_id: YouTube video ID

        Returns:
            Transcript text or None if extraction fails
        """
        if not video_id or not isinstance(video_id, str):
            print("Invalid video ID provided")
            return None

        try:
            transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
            if not transcript_list:
                print(f"No transcript found for video {video_id}")
                return None

            # Ensure proper formatting with punctuation
            formatted_segments = []
            for segment in transcript_list:
                text = segment.get('text', '').strip()
                if text:
                    # Add period if segment doesn't end with punctuation
                    if not text[-1] in ['.', '!', '?', ':', ';']:
                        text += '.'
                    formatted_segments.append(text)

            full_transcript = ' '.join(formatted_segments)
            return full_transcript
        except Exception as e:
            print(f"Error fetching transcript for video {video_id}: {e}")
            return None

    def simple_tokenize(self, text):
        """Simple tokenizer that avoids NLTK's punkt."""
        text = re.sub(r'[^\w\s]', ' ', text)
        return [token for token in text.lower().split() if token]

    def preprocess_text(self, text):
        """NLP preprocessing: stopword removal, lemmatization."""
        if not text or not isinstance(text, str):
            return "", {}

        text = re.sub(r'[^\w\s]', ' ', text).lower()
        tokens = self.simple_tokenize(text)
        filtered_tokens = [word for word in tokens if word not in self.stop_words]

        # Use lemmatizer if available
        if self.lemmatizer:
            lemmatized_tokens = [self.lemmatizer.lemmatize(word) for word in filtered_tokens]
        else:
            lemmatized_tokens = filtered_tokens

        processed_text = ' '.join(lemmatized_tokens)
        return processed_text, {}

    def embed_query(self, query: str) -> List[float]:
        """Generate embedding for the query."""
        if not query or not isinstance(query, str):
            print("Warning: Empty or invalid query received for embedding")
            return [0.0] * self.embedding_model.get_sentence_embedding_dimension()

        # Apply preprocessing
        processed_query, _ = self.preprocess_text(query)

        # Ensure we have text to embed
        if not processed_query:
            processed_query = query  # Fall back to original query

        # Generate embedding
        return self.embedding_model.encode(processed_query).tolist()

    def retrieve_relevant_chunks(self, query_embedding: List[float], top_k: int = 1, video_id: Optional[str] = None) -> List[Dict[str, Any]]:
        """
        Retrieve the most relevant chunk from Pinecone based on the query embedding.
        Optionally filter by video ID.
        """
        # Prepare query parameters
        query_params = {
            "vector": query_embedding,
            "top_k": top_k,
            "include_metadata": True
        }

        # Add video ID filter if provided
        if video_id:
            query_params["filter"] = {"video_id": video_id}

        # Query Pinecone index using similarity search
        try:
            query_response = self.index.query(**query_params)
        except Exception as e:
            print(f"Error querying Pinecone: {e}")
            return []

        # Extract matches with their metadata
        matches = query_response.get('matches', [])

        # Format results
        results = []
        for match in matches:
            # Extract text sample from metadata if available
            text = match.metadata.get('text_sample', 'No text available')

            # Format the result
            result = {
                'id': match.id,
                'score': match.score,
                'text': text,
                'video_id': match.metadata.get('video_id', 'unknown'),
                'chunk_id': match.metadata.get('chunk_id', -1)
            }
            results.append(result)

        return results

    def query(self, question: str, video_id: Optional[str] = None) -> Dict[str, Any]:
        """
        Process a user query and return a well-structured answer with supporting evidence.

        Args:
            question: The user's question
            video_id: Optional YouTube video ID to filter sources
        """
        # Step 1: Generate embedding for the question
        query_embedding = self.embed_query(question)

        # Step 2: Retrieve the most relevant chunk from Pinecone, optionally filtered by video ID
        relevant_chunks = self.retrieve_relevant_chunks(
            query_embedding,
            self.top_k,
            video_id
        )

        # If no chunks found and a video ID was provided, try without the filter
        if not relevant_chunks and video_id:
            relevant_chunks = self.retrieve_relevant_chunks(query_embedding, self.top_k)

        # Step 3: Generate a comprehensive answer based on the retrieved chunk
        answer = self.generate_answer(question, relevant_chunks)

        # Step 4: Try to fetch video transcript if a video ID was provided and no relevant chunks found
        video_transcript = None
        if video_id and not relevant_chunks:
            video_transcript = self.extract_transcript(video_id)

        # Step 5: Return the answer and top source
        result = {
            "question": question,
            "answer": answer,
            "source": relevant_chunks[0] if relevant_chunks else None,
            "video_id": video_id,
            "video_transcript": video_transcript
        }

        return result

    def format_response(self, result: Dict[str, Any]) -> str:
        """
        Format the query result into a well-structured response string.
        """
        response = f"QUESTION: {result['question']}\n\nANSWER:\n{result['answer']}\n\n"

        if result.get('source'):
            source = result['source']
            response += "TOP SOURCE:\n"
            response += f"Score: {source['score']:.2f}\n"
            response += f"Text: {source['text']}\n"

        # Add video transcript if available
        if result.get('video_transcript'):
            response += "\nVIDEO TRANSCRIPT EXCERPT:\n"
            # Limit transcript to first 500 characters
            transcript_excerpt = result['video_transcript'][:500] + "..."
            response += transcript_excerpt + "\n"

        return response


# Example usage
def main():
    # Initialize the RAG system
    pinecone_api_key = "pcsk_7EKroD_MaZi2zjikyZTdpaDPCkit4qEAE6cjKuJ7C2ot9htS7EE6uurWQLrfznykMd7bW3"
    groq_api_key = "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"
    index_name = "embeddings"

    print("Initializing Enhanced RAG system...")
    rag = EnhancedRAGSystem(
        pinecone_api_key=pinecone_api_key,
        groq_api_key=groq_api_key,
        index_name=index_name,
        model_name="sentence-transformers/all-mpnet-base-v2"
    )

    # Example query with video ID
    question = "What is JVM?"
    video_id = "NUy_wOxOM8E"  # Example YouTube video ID

    # Add error handling around the main query operation
    try:
        print(f"Processing query: '{question}' with video ID: {video_id}")
        result = rag.query(question, video_id)

        # Format and print the result
        formatted_response = rag.format_response(result)
        print("\n" + "="*50)
        print(formatted_response)
        print("="*50)
    except Exception as e:
        print(f"An error occurred during query processing: {e}")

if __name__ == "__main__":
    main()

Initializing Enhanced RAG system...
Using device: cpu
Processing query: 'What is JVM?' with video ID: NUy_wOxOM8E

QUESTION: What is JVM?

ANSWER:
**What is JVM (Java Virtual Machine)?**

As we embark on our journey to master Java, it's essential to establish a solid foundation in Java terminology. One of the critical concepts to grasp is the Java Virtual Machine, commonly referred to as JVM.

**Definition:**
The Java Virtual Machine (JVM) is a virtual machine that runs Java bytecode on a computer. It's the runtime environment for Java, responsible for executing Java programs.

**How it works:**
When you compile a Java program, the Java compiler (javac) converts the Java source code (.java files) into an intermediate format called bytecode (.class files). This bytecode is platform-independent, meaning it can run on any device supporting a JVM, regardless of the underlying operating system or hardware architecture.

The JVM interprets and executes the bytecode, providing a layer of abst